# Data reduction for Amor

In this notebook, we will look at the reduction workflow for reflectometry data collected from the PSI
[Amor](https://www.psi.ch/en/sinq/amor) instrument.
This is a living document and there are plans to update this as necessary with changes in the data reduction methodology and code.

We will begin by importing the modules that are necessary for this notebook and loading the data.
The `sample.nxs` file is the experimental data file of interest,
while `reference.nxs` is the reference measurement of the neutron supermirror.

In [ ]:
import scipp as sc
from ess import amor

## The Amor beamline

Before we can load the data, we need to define the parameters of the beamline.
We begin by defining the convention for naming angles in our set-up.
We use the Fig. 5 from the paper by [Stahn & Glavic (2016)](https://doi.org/10.1016/j.nima.2016.03.007),
which is reproduced below (along with its caption).

![Figure5](https://ars.els-cdn.com/content/image/1-s2.0-S0168900216300250-gr5.jpg)

The yellow area shows the incoming and reflected beam, both with the divergence $\Delta \theta$.
The inclination of the sample relative to the centre of the incoming beam
(here identical to the instrument horizon) is called $\omega$,
and the respective angle of the reflected beam relative to the same axis is $\gamma$.

In general the detector centre is located at $\gamma_{\rm D} = 2\omega$.
These are instrument coordinates and should not be confused with the situation on the sample,
where the take-off angle of an individual neutron trajectory is called $\theta$.

The `amor` module provides a helper function that generates the default beamline parameters.
This function requires the sample rotation angle ($\omega$) as an input to fully define the beamline.
In the future, all these beamline parameters (including the sample rotation) will be included in the file meta data.
For now, we must define this manually.

In [ ]:
sample_rotation = sc.scalar(0.7989, unit='deg')
amor_beamline = amor.make_beamline(sample_rotation=sample_rotation)

## Loading the data

Using the `amor.load` function, we load the `sample.nxs` file and perform some early preprocessing:

- The `tof` values are converted from nanoseconds to microseconds.
- The raw data contains events coming from two pulses, and these get folded into a single `tof` range

In [ ]:
sample = amor.load(amor.data.get_path("sample.nxs"),
                   beamline=amor_beamline)
sample

By simply plotting the data, we get a first glimpse into the data contents.

In [ ]:
sc.plot(sample)

### Correcting the position of the detector pixels

**Note:** once new Nexus files are produced, this step should go away. 

The pixel positions are wrong in the `sample.nxs` file, and require an ad-hoc correction.
We apply an arbitrary shift in the vertical (`y`) direction.
We first move the pixels down by 0.955 degrees,
so that the centre of the beam goes through the centre of the top half of the detector blades
(the bottom half of the detectors was turned off).
Next, we move all the pixels so that the centre of the top half of the detector pixels lies at an angle of $2 \omega$,
as described in the beamline diagram.

In [ ]:
def pixel_position_correction(data: sc.DataArray):
    return data.coords['position'].fields.z * sc.tan(2.0 *
                                                     data.coords['sample_rotation'] -
                                                     (0.955 * sc.units.deg))
sample.coords['position'].fields.y += pixel_position_correction(sample)

We now check that the detector pixels are in the correct position by showing the instrument view

In [ ]:
amor.instrument_view(sample)

## Coordinate transformation graph

To compute the wavelength $\lambda$, the scattering angle $\theta$, and the $Q$ vector for our data,
we construct a coordinate transformation graph.

It is based on classical conversions from `tof` and pixel `position` to $\lambda$ (`wavelength`),
$\theta$ (`theta`) and $Q$ (`Q`),
but comprises a number of modifications.

The computation of the scattering angle $\theta$ includes a correction for the Earth's gravitational field which bends the flight path of the neutrons.
The angle can be found using the following expression

$$\theta = \sin^{-1}\left(\frac{\left\lvert y + \frac{g m_{\rm n}}{2 h^{2}} \lambda^{2} L_{2}^{2} \right\rvert }{L_{2}}\right) - \omega$$

where $m_{\rm n}$ is the neutron mass,
$g$ is the acceleration due to gravity,
and $h$ is Planck's constant
(this assumes the neutrons are all travelling horizontally at the position of the sample).

This expression differs slightly from the equation found in the computation of the $\theta$ angle in other techniques such as
[SANS](https://docs.mantidproject.org/v3.9.0/algorithms/Q1D-v2.html#algm-q1d),
in that the horizontal $x$ term is absent,
because we assume a planar symmetry and only consider the vertical $y$ component of the displacement.

The conversion graph is defined in the reflectometry module,
and can be obtained via

In [ ]:
graph = amor.conversions.specular_reflection_graph()
sc.show_graph(graph, simplified=True)

## Computing the wavelength

To compute the wavelength of the neutrons,
we request the `wavelength` coordinate from the `transform_coords` method by supplying our graph defined above
(see [here](https://scipp.github.io/scippneutron/user-guide/coordinate-transformations.html)
for more information about using `transform_coords`).

We also exclude all neutrons with a wavelength lower than 2.4 &#8491;.

In [ ]:
sample_wav = sample.transform_coords(["wavelength"], graph=graph)
wavelength_edges = sc.array(dims=['wavelength'], values=[2.4, 16.0], unit='angstrom')
sample_wav = sc.bin(sample_wav, edges=[wavelength_edges])
sample_wav

In [ ]:
sample_wav.bins.concatenate('detector_id').plot()

## Compute the Q vector

Using the same method, we can compute the $Q$ vector,
which now depends on both detector position (id) and wavelength

In [ ]:
sample_q = sample_wav.transform_coords(["Q"], graph=graph)
sample_q

In [ ]:
q_edges = sc.geomspace(dim='Q', start=0.008, stop=0.08, num=201, unit='1/angstrom')
sample_q_binned = sc.bin(sample_q, edges=[q_edges])
sample_q_summed = sample_q_binned.sum('detector_id')
sc.plot(sample_q_summed["wavelength", 0], norm="log")

## Normalize by the super-mirror

To perform the normalization, we load the super-mirror `reference.nxs` file.

In [ ]:
reference = amor.load(amor.data.get_path("reference.nxs"),
                      beamline=amor_beamline)
reference.coords['position'].fields.y += pixel_position_correction(reference)
reference

We convert the reference to wavelength using the same graph

In [ ]:
reference_wav = reference.transform_coords(["wavelength"], graph=graph)
reference_wav = sc.bin(reference_wav, edges=[wavelength_edges])
reference_wav.bins.concatenate('detector_id').plot()

And we then convert to $Q$ as well

In [ ]:
reference_q = reference_wav.transform_coords(["Q"], graph=graph)
reference_q_binned = sc.bin(reference_q, edges=[q_edges])
reference_q_summed = reference_q_binned.sum('detector_id')
sc.plot(reference_q_summed["wavelength", 0], norm="log")

Finally, we divide the sample by the reference to obtain

In [ ]:
normalized = sample_q_summed["wavelength", 0] / reference_q_summed["wavelength", 0]
sc.plot(normalized, norm="log")

## Make a $(\lambda, \theta)$ map

A good sanity check is to create a two-dimensional map of the counts in $\lambda$ and $\theta$ bins.
To achieve this, we request two output coordinates from the `transform_coords` method.

In [ ]:
sample_theta = sample.transform_coords(["theta", "wavelength"], graph=graph)
sample_theta

Then, we concatenate all the events in the `detector_id` dimension

In [ ]:
sample_theta = sample_theta.bins.concatenate('detector_id')
sample_theta

Finally, we bin into the existing `theta` dimension, and into a new `wavelength` dimension,
to create a 2D output

In [ ]:
nbins = 165
theta_edges = sc.linspace(dim='theta', start=0.4, stop=1.2, num=nbins, unit='deg')
wavelength_edges = sc.linspace(dim='wavelength', start=1.0, stop=15.0, num=nbins, unit='angstrom')
binned = sc.bin(sample_theta, edges=[sc.to_unit(theta_edges, 'rad'), wavelength_edges])
binned

In [ ]:
binned.bins.sum().plot()

This plot can be used to check if the value of the sample rotation angle $\omega$ is correct.
The bright triangles should be pointing back to the origin $\lambda = \theta = 0$.